In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import datetime 

In [ ]:
circuits = pd.read_csv('circuits.csv')
circuits.head()

,circuitid,circuitref,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park


In [ ]:
laptimes = pd.read_csv('lap_times.csv')
laptimes.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [ ]:
pitstops = pd.read_csv('pit_stops.csv')
pitstops.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [ ]:
seasons = pd.read_csv('seasons.csv',parse_dates=['year'])
seasons.head()

,year,url
0,2009-01-01,http://en.wikipedia.org/wiki/2009_Formula_One_...
1,2008-01-01,http://en.wikipedia.org/wiki/2008_Formula_One_...
2,2007-01-01,http://en.wikipedia.org/wiki/2007_Formula_One_...
3,2006-01-01,http://en.wikipedia.org/wiki/2006_Formula_One_...
4,2005-01-01,http://en.wikipedia.org/wiki/2005_Formula_One_...


In [ ]:
status = pd.read_csv('status.csv')
status.head()

,statusid,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [ ]:
constructor_standings = pd.read_csv('constructor_standings.csv')
constructors = pd.read_csv('constructors.csv')
driver_standings = pd.read_csv('driver_standings.csv')
drivers = pd.read_csv('drivers.csv')

In [ ]:
races = pd.read_csv('races.csv',parse_dates=['year'])
constructor_results = pd.read_csv('constructor_results.csv')
results = pd.read_csv('results.csv')
qualifying = pd.read_csv('qualifying.csv')


Most successful constructors
Many well-known, established names have competed at the highest level of racing with little success. A good reason for this is that F1 is not the best use of value creation, according to car makers, as the costs and returns associated with running an F1 team are close to breaking even at best. Good old finance says that any investment's return on equity being less than the cost of capital is a venture that destroys value. But this might not be entirely true in all cases as F1 provides a great platform to advertise and market the respective carmaker's brand; it's a cost that could be incurred to increase the top-line growth. Let's look at two charts to understand which constructors have consistently put in the money and engineering prowess to stay at the top. i.e. if you're a mega fan of the sport, you already know the answer.

#merging the constructors dataframe with race results

In [ ]:
team = constructors.merge(results,on='constructorId',how = 'left')

In [ ]:

best = team[['name','points','raceId']]
best = best.groupby('name')['raceId'].nunique().sort_values(ascending=False).reset_index(name = 'races')
best = best[best['races'] >= 100]
best.head() 

,name,races
0,Ferrari,1043
1,McLaren,872
2,Williams,786
3,Tyrrell,433
4,Renault,403


In [ ]:
func = lambda x: x.points.sum()/x.raceId.nunique()
data = team[team['name'].isin(best.name)].groupby('name').apply(func).sort_values(ascending=False).reset_index(name = 'points_per_race')
data.head(10)

,name,points_per_race
0,Mercedes,25.611538
1,Red Bull,17.667656
2,Ferrari,9.479386
3,McLaren,6.961009
4,Force India,5.179245
5,Williams,4.569975
6,Renault,4.409429
7,Benetton,3.311538
8,BRM,2.581731
9,Team Lotus,2.518987


In [ ]:
fig = go.Figure(
    data=[go.Bar(x = data.name, y=data['points_per_race'])],
    layout_title_text="Constructor's Points per Race"
    
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [ ]:
historic_points = team.groupby('name').agg({'points':'sum'}).sort_values('points',ascending=False).reset_index().head(10)
historic_points

,name,points
0,Ferrari,9887.0
1,Mercedes,6659.0
2,McLaren,6070.0
3,Red Bull,5954.0
4,Williams,3592.0
5,Renault,1777.0
6,Force India,1098.0
7,Team Lotus,995.0
8,Benetton,861.0
9,Tyrrell,711.0


In [ ]:
fig = go.Figure(
    data=[go.Bar(x = historic_points.name, y=historic_points['points'])],
    layout_title_text="Constructor's Historic Points"
)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

In [1]:
df = circuits.merge(races,how='left',left_on = 'circuitid',right_on = 'circuitId')
df2 = df.merge(results,how='left',on='raceId')
status_df = df2.merge(status,how='inner',left_on = 'statusId',right_on= 'statusid')

NameError: ignored

In [ ]:
status_df.drop(['name_y','url_y','url_x','time_y'],axis=1,inplace=True)
status_df.rename(columns={'name_x':'name','time_x':'time'},inplace=True)

In [ ]:
altitude = status_df[status_df['status'].isin(['Transmission','Engine','Overheating'])]
altitude = altitude[altitude['year'] >= pd.to_datetime('2015-01-01')]
altitude.head()

,circuitid,circuitref,name,location,country,lat,lng,alt,raceId,year,...,points,laps,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,statusid,status
14065,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,926,2015-01-01,...,0.0,32.0,NaN,30.0,13.0,01:34.3,202.458,5.0,5,Engine
14066,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,948,2016-01-01,...,0.0,38.0,NaN,15.0,21.0,01:33.9,203.327,5.0,5,Engine
14067,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,948,2016-01-01,...,0.0,21.0,NaN,21.0,5.0,01:30.7,210.480,5.0,5,Engine
14068,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,989,2018-01-01,...,0.0,13.0,NaN,13.0,18.0,01:30.6,210.601,5.0,5,Engine
14069,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,1010,2019-01-01,...,0.0,9.0,NaN,9.0,20.0,01:30.9,210.022,5.0,5,Engine


In [ ]:
circuit_altitudes = altitude.groupby(['name','alt'])['status'].count().sort_values(ascending = False).reset_index().head(10)
circuit_altitudes.rename(columns={'status':'engine & transmission failures'},inplace=True)
circuit_altitudes

,name,alt,engine & transmission failures
0,Autódromo Hermanos Rodríguez,2227,7
1,Bahrain International Circuit,7,7
2,Red Bull Ring,678,6
3,Albert Park Grand Prix Circuit,10,5
4,Autodromo Nazionale di Monza,162,5
5,Circuit de Spa-Francorchamps,401,4
6,Marina Bay Street Circuit,18,4
7,Yas Marina Circuit,3,4
8,Circuit de Barcelona-Catalunya,109,3
9,Hockenheimring,103,3


In [ ]:
#plotting a bubble chart: bigger the size of the bubble, higher the altitude

df = circuit_altitudes

fig = px.scatter(df, x="alt", y="engine & transmission failures",
         size="alt", color="name",
                  log_x=True, size_max= 80)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
# grouping by nationality, counting the driver and plotting a pie chart

driver_nationality = drivers.groupby('nationality')['nationality'].count().sort_values(ascending = False).reset_index(name = 'number of drivers')
fig = go.Figure(data=[go.Pie(labels=driver_nationality.nationality.head(10), values=driver_nationality['number of drivers'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Historical Driver Nationality Distribution since 1950")
fig.show()

In [ ]:
#merging drivers, driver standings and race data 

driver_position = drivers.merge(driver_standings,left_on='driverid',right_on='driverId',how = 'left')
driver_position = driver_position.merge(races,on = 'raceId',how = 'left')

In [ ]:
#grouping by nationality year and surname to get the max points achieved every season

champions = driver_position.groupby(['nationality','year','surname'])[['points','wins']
                                            ].max().sort_values('points',ascending = False).reset_index()
champions.drop_duplicates(subset=['year'], inplace=True)

In [ ]:
champion_nations = champions.nationality.value_counts().to_frame()
fig = go.Figure(data=[go.Pie(labels=champion_nations.index, values=champion_nations['nationality'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Distribution of Historic Champions by Nation")
fig.show()

In [ ]:
#grouping by nationality year and surname to get the max points achieved every season and dropping year duplicates

champion_drivers = driver_position.groupby(['nationality','year','surname'])[['points','wins']
                                            ].max().sort_values('points',ascending = False).reset_index()
champion_drivers.drop_duplicates(subset=['year'], inplace=True)

#grouping by nationality and counting the surname of drivers 

final = champion_drivers.groupby('nationality')['surname'].nunique().reset_index(name = 'champions').sort_values(
    by='champions',ascending = False)

#merging both the datasets and creating a column to calculate the ratio

ratios = final.merge(driver_nationality,on='nationality',how='inner')
ratios['perc_winners'] = (ratios.champions/ratios['number of drivers']*100).round(2)
ratios = ratios.sort_values('perc_winners',ascending = False)
ratios.head(5) 

,nationality,champions,number of drivers,perc_winners
2,Finnish,3,9,33.33
6,Austrian,2,15,13.33
5,Australian,2,17,11.76
12,New Zealander,1,9,11.11
1,Brazilian,3,32,9.38


In [ ]:
#creating a bar chart

df = ratios
fig = px.bar(df, x='nationality', y='perc_winners',
         hover_data=['champions','number of drivers'], color='number of drivers',
         height=400)
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Champions from a nation with respect to total drivers from the nation")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


In [ ]:
#merging driver data, their standings and race data

driver_position = drivers.merge(driver_standings,left_on='driverid',right_on='driverId',how = 'left')
driver_position = driver_position.merge(races,on = 'raceId',how = 'left')

In [ ]:
#filtering the dataset to include only where the position is 1 and grouping by name, year and extracting the max wins

positions = driver_position[driver_position['position'] == 1].groupby(
    ['surname','year'])['wins'].max().sort_values(ascending=False).reset_index(name = 'Wins')
positions.head(20)
positions.year = positions.year.dt.year
positions.rename(columns={'surname':'name'},inplace=True)
positions.Wins = positions.Wins.astype('int64')

positions.head(20)

,name,year,Wins
0,Schumacher,2004,13
1,Vettel,2013,13
2,Hamilton,2019,11
3,Vettel,2011,11
4,Schumacher,2002,11
5,Hamilton,2020,11
6,Hamilton,2018,11
7,Hamilton,2014,11
8,Hamilton,2015,10
9,Verstappen,2021,10


In [ ]:
#plotting a bubble chart

fig = px.scatter(positions.head(30), x="year", y="Wins", color="name",
                 title="Most wins by a driver in a single season",size = 'Wins')
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.show()

In [ ]:
competition = driver_position[driver_position['year'].dt.year == 1991
               ].groupby(['surname','year']).points.max().sort_values(ascending = False).reset_index().head(5)
competition.year = competition.year.dt.year
competition.iloc[0:2,0]

0      Senna
1    Mansell
Name: surname, dtype: object

In [ ]:
def rivalry(x):
    competition = driver_position[driver_position['year'].dt.year == x
               ].groupby(['surname','year']).points.max().sort_values(ascending = False).reset_index().head(5)
    competition.year = competition.year.dt.year    
    
    h = driver_position.merge(circuits,left_on='circuitId',right_on='circuitid',how = 'left')
    h.rename(columns={'name_y':'circuit_name'},inplace=True)
    viz = h.loc[:,['date','year','circuit_name','surname','points','wins']]

    viz.dropna(inplace = True)

    viz.points = viz.points.astype('int64')
    viz.wins = viz.wins.astype('int64')
    viz.year = viz.year.dt.year
    viz.date = pd.to_datetime(viz.date)
    
    top_five = viz[viz.loc[:,'year'] == x]
    top_five = top_five.groupby(['surname'])[['points','wins']].max().sort_values('points',ascending = False).head(6).reset_index()
    
    duo = competition.iloc[0:2]
    if competition.iloc[0,2] - competition.iloc[1,2] <= 10:
        print('\033[1m' + 'A rivalry in the history books!')
    elif competition.iloc[0,2] - competition.iloc[1,2] <= 20:
        print('\033[1m' + 'Spicy!')
    elif competition.iloc[0,2] - competition.iloc[1,2] < 30:
        print('\033[1m' + 'Meh!')
    elif competition.iloc[0,2] - competition.iloc[1,2] >= 30:
        print('\033[1m' + 'Snore Fest!')

        
    df = top_five
    fig = px.bar(df, x='surname', y='points',
             hover_data=['wins'], color='points',
            height=400,color_continuous_scale= 'turbo')
    fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    
    print('----------------------------------')
    x = fig.show()
    return x

In [ ]:
rivalry(2022)  

Snore Fest!
----------------------------------


In [ ]:
#merging and extraction of important columns

fast = circuits.merge(races,left_on = 'circuitid',right_on='circuitId',how = 'left')
fast = fast.merge(results,on='raceId',how = 'left')
fast = fast.merge(drivers,left_on='driverId',right_on='driverid',how = 'inner')
fast.rename(columns={'name_x':'circuit_name'},inplace = True)
fast = fast[['circuit_name','country','surname','fastestLapTime','nationality','year']]

# dropping null values and converting fastestlaptime into seconds

fast = fast.dropna()
fast['fastestLapTime_seconds']=fast['fastestLapTime'].apply(lambda x: float(x.split(':')[0])*60+float(x.split(':')[1])) 

In [ ]:
fast['fastest_recorded_lap'] = fast.groupby(['circuit_name'])['fastestLapTime'].transform('min')
fastest = fast[fast['fastest_recorded_lap']==fast['fastestLapTime']].sort_values('country').reset_index(col_level = 1)
fastest.drop(['index','fastest_recorded_lap'],inplace=True,axis = 1)
fastest['year'] = fastest.year.dt.year
fastest

,circuit_name,country,surname,fastestLapTime,nationality,year,fastestLapTime_seconds
0,Albert Park Grand Prix Circuit,Australia,Leclerc,01:20.3,Monegasque,2022,80.3
1,Red Bull Ring,Austria,Sainz,01:05.6,Spanish,2020,65.6
2,Baku City Circuit,Azerbaijan,Leclerc,01:43.0,Monegasque,2019,103.0
3,Bahrain International Circuit,Bahrain,Russell,00:55.4,British,2020,55.4
4,Circuit de Spa-Francorchamps,Belgium,Räikkönen,01:45.1,Finnish,2004,105.1
5,Autódromo José Carlos Pace,Brazil,Bottas,01:10.5,Finnish,2018,70.5
6,Circuit Gilles Villeneuve,Canada,Bottas,01:13.1,Finnish,2019,73.1
7,Shanghai International Circuit,China,Schumacher,01:32.2,German,2004,92.2
8,Circuit Paul Ricard,France,Vettel,01:32.7,German,2019,92.7
9,Circuit de Nevers Magny-Cours,France,Schumacher,01:15.4,German,2004,75.4


In [ ]:
fastest_viz = fastest.surname.value_counts().rename_axis('driver').reset_index(name= 'fastest laps') 

df = fastest_viz
fig = px.bar(df, x='driver', y='fastest laps',
         hover_data=['fastest laps'], color='fastest laps',
        height=400,color_continuous_scale= 'Blues')
fig.update_layout(
    title="Drivers with the most fastest ever laps")
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

In [ ]:
#merging the (1) two dataframes [drivers and qualifying] needed and (2) performing aggregation function

driver_quali = results.merge(drivers,left_on = 'driverId',right_on='driverid',how = 'left')
driver_quali['full_name'] = driver_quali['forename'] + ' ' + driver_quali['surname']
driver_quali = driver_quali[['full_name','grid','position']]


quali_wins = driver_quali[driver_quali['grid'] == 1].groupby('full_name')['grid'].sum().reset_index(
                                        name = 'grid poles').sort_values('grid poles',ascending = False)

#resetting index numbers 

quali_wins = quali_wins.reset_index(col_level=0)
quali_wins.drop('index',axis = 1,inplace = True)
quali_wins.head(20)

,full_name,grid poles
0,Lewis Hamilton,103
1,Michael Schumacher,68
2,Ayrton Senna,65
3,Sebastian Vettel,57
4,Jim Clark,34
5,Alain Prost,33
6,Nigel Mansell,32
7,Nico Rosberg,30
8,Juan Fangio,29
9,Mika Häkkinen,26


In [ ]:
#merging (1) dataframes [drivers, results], (2) creating a full name column and (3) selecting the important columns 

race_wins = drivers.merge(results,left_on='driverid',right_on='driverId',how = 'left')
race_wins['full_name'] = race_wins['forename'] + ' ' + race_wins['surname']

race_wins = race_wins[['full_name','position']]

# grouping by surname and counting the number of races won

highest_rw = race_wins[race_wins['position'] == 1.0].groupby('full_name').count().sort_values(
    'position', ascending = False).reset_index()
highest_rw.head(20)

,full_name,position
0,Lewis Hamilton,103
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Ayrton Senna,41
5,Fernando Alonso,32
6,Nigel Mansell,31
7,Jackie Stewart,27
8,Max Verstappen,26
9,Niki Lauda,25


In [ ]:
#merging the race wins alongside starting at grid position 1 

racexpole = highest_rw.merge(quali_wins,on = 'full_name',how = 'left')
racexpole = racexpole[racexpole['grid poles'] > 10]                    #setting minimum grid poles as > 10 
racexpole.dropna(inplace=True)                                         #dropping 28 null values

#cosmetic changes and calculation of racexgrid pole ratio

racexpole.rename(columns={'position':'race poles'},inplace=True)
racexpole['grid poles'] = racexpole['grid poles'].astype(int)
racexpole['racexgrid'] = (racexpole['race poles']/racexpole['grid poles']).round(2)
racexpole = racexpole.sort_values('racexgrid',ascending=False).reset_index(col_level=0)
racexpole.drop('index',axis = 1,inplace = True)
racexpole.head(15)

,full_name,race poles,grid poles,racexgrid
0,Max Verstappen,26,16,1.62
1,Jackie Stewart,27,17,1.59
2,Alain Prost,51,33,1.55
3,Fernando Alonso,32,22,1.45
4,Michael Schumacher,91,68,1.34
5,Kimi Räikkönen,21,18,1.17
6,Damon Hill,22,20,1.10
7,David Coulthard,13,12,1.08
8,Jack Brabham,14,13,1.08
9,Graham Hill,14,13,1.08


In [ ]:
df = racexpole.head(12)
fig = px.bar(df, x='full_name', y='racexgrid',
         hover_data=['race poles','grid poles'], color='racexgrid',
        height=400,color_continuous_scale= 'gray')
fig.update_layout(
    title="Race Poles/Grid Poles Ratio")
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

In [ ]:
tracks = circuits.merge(races, left_on='circuitid',right_on='circuitId',how = 'inner') 
tracks = tracks.merge(results,on = 'raceId',how = 'left')
tracks = tracks[['name_x','circuitid','driverId','position','grid','raceId','year']]
tracks.rename(columns={'name_x':'circuit'},inplace=True)
tracks.dropna(inplace = True)


tracks = tracks.loc[(tracks['year'] >= '2010-01-01')]
tracks

,circuit,circuitid,driverId,position,grid,raceId,year
302,Albert Park Grand Prix Circuit,1,18.0,1.0,4.0,338,2010-01-01
303,Albert Park Grand Prix Circuit,1,9.0,2.0,9.0,338,2010-01-01
304,Albert Park Grand Prix Circuit,1,13.0,3.0,5.0,338,2010-01-01
305,Albert Park Grand Prix Circuit,1,4.0,4.0,3.0,338,2010-01-01
306,Albert Park Grand Prix Circuit,1,3.0,5.0,6.0,338,2010-01-01
...,...,...,...,...,...,...,...
25623,Miami International Autodrome,79,817.0,13.0,14.0,1078,2022-01-01
25624,Miami International Autodrome,79,849.0,14.0,19.0,1078,2022-01-01
25625,Miami International Autodrome,79,854.0,15.0,15.0,1078,2022-01-01
25626,Miami International Autodrome,79,825.0,16.0,16.0,1078,2022-01-01


In [ ]:
tracks['position_status'] = np.where(tracks['position'] == tracks['grid'],1,0)
tracks

,circuit,circuitid,driverId,position,grid,raceId,year,position_status
302,Albert Park Grand Prix Circuit,1,18.0,1.0,4.0,338,2010-01-01,0
303,Albert Park Grand Prix Circuit,1,9.0,2.0,9.0,338,2010-01-01,0
304,Albert Park Grand Prix Circuit,1,13.0,3.0,5.0,338,2010-01-01,0
305,Albert Park Grand Prix Circuit,1,4.0,4.0,3.0,338,2010-01-01,0
306,Albert Park Grand Prix Circuit,1,3.0,5.0,6.0,338,2010-01-01,0
...,...,...,...,...,...,...,...,...
25623,Miami International Autodrome,79,817.0,13.0,14.0,1078,2022-01-01,0
25624,Miami International Autodrome,79,849.0,14.0,19.0,1078,2022-01-01,0
25625,Miami International Autodrome,79,854.0,15.0,15.0,1078,2022-01-01,1
25626,Miami International Autodrome,79,825.0,16.0,16.0,1078,2022-01-01,1


In [ ]:
#counting total number of races held in each track and attaching it to the previously created tracks column

total_races = races.loc[(races['year'] >= '2010-01-01')]
total_races = total_races.circuitId.value_counts().reset_index()
total_races.rename(columns={"index":'circuitid','circuitId':'num_races'},inplace=True)
tracks = tracks.merge(total_races,on='circuitid',how = 'left')  
tracks

,circuit,circuitid,driverId,position,grid,raceId,year,position_status,num_races
0,Albert Park Grand Prix Circuit,1,18.0,1.0,4.0,338,2010-01-01,0,11
1,Albert Park Grand Prix Circuit,1,9.0,2.0,9.0,338,2010-01-01,0,11
2,Albert Park Grand Prix Circuit,1,13.0,3.0,5.0,338,2010-01-01,0,11
3,Albert Park Grand Prix Circuit,1,4.0,4.0,3.0,338,2010-01-01,0,11
4,Albert Park Grand Prix Circuit,1,3.0,5.0,6.0,338,2010-01-01,0,11
...,...,...,...,...,...,...,...,...,...
4384,Miami International Autodrome,79,817.0,13.0,14.0,1078,2022-01-01,0,1
4385,Miami International Autodrome,79,849.0,14.0,19.0,1078,2022-01-01,0,1
4386,Miami International Autodrome,79,854.0,15.0,15.0,1078,2022-01-01,1,1
4387,Miami International Autodrome,79,825.0,16.0,16.0,1078,2022-01-01,1,1


In [ ]:
#pivot table to find the count of unchanged positions [marked numbers column 1 values]

circuit_rating = tracks[tracks['num_races'] >= 5].pivot_table(index=['circuit','num_races'], columns='position_status', aggfunc='size', 
                                             fill_value=0)
circuit_rating = circuit_rating.reset_index()
circuit_rating

position_status,circuit,num_races,0,1
0,Albert Park Grand Prix Circuit,11,146,18
1,Autodromo Nazionale di Monza,13,185,30
2,Autódromo Hermanos Rodríguez,7,84,20
3,Autódromo José Carlos Pace,12,169,33
4,Bahrain International Circuit,13,205,25
5,Baku City Circuit,6,81,12
6,Circuit Gilles Villeneuve,11,166,30
7,Circuit de Barcelona-Catalunya,13,199,41
8,Circuit de Monaco,12,162,35
9,Circuit de Spa-Francorchamps,13,180,34


In [ ]:
#checking for tracks that had the least amount of overtaking action
circuit_rating['boring_score'] = circuit_rating[1].divide(circuit_rating['num_races'])
circuit_rating = circuit_rating.sort_values('boring_score',ascending=False)

circuit_rating

position_status,circuit,num_races,0,1,boring_score
20,Yas Marina Circuit,13,170,48,3.692308
11,Hockenheimring,6,91,19,3.166667
7,Circuit de Barcelona-Catalunya,13,199,41,3.153846
10,Circuit of the Americas,10,123,30,3.000000
8,Circuit de Monaco,12,162,35,2.916667
13,Marina Bay Street Circuit,11,142,32,2.909091
2,Autódromo Hermanos Rodríguez,7,84,20,2.857143
16,Shanghai International Circuit,10,165,28,2.800000
3,Autódromo José Carlos Pace,12,169,33,2.750000
6,Circuit Gilles Villeneuve,11,166,30,2.727273


In [ ]:
df = circuit_rating.head(10)
fig = px.bar(df, x='circuit', y='boring_score',
         hover_data=['num_races',1], color='num_races',
        height=400,color_continuous_scale= 'ice')
fig.update_layout(
    title="Worst Tracks to Overtake")
fig.update_traces(textfont_size=20,
              marker=dict(line=dict(color='#000000', width=2)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)